In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version

spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:4 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [ ]:
#  get the postgres driver
!wget https://jdbc.postgresql.org/download/postgresql-42.3.1.jar

--2022-01-19 22:16:56--  https://jdbc.postgresql.org/download/postgresql-42.3.1.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1015689 (992K) [application/java-archive]
Saving to: ‘postgresql-42.3.1.jar.2’

postgresql-42.3.1.j 100%[===================>] 991.88K  2.06MB/s    in 0.5s    

2022-01-19 22:16:57 (2.06 MB/s) - ‘postgresql-42.3.1.jar.2’ saved [1015689/1015689]



In [ ]:
#create a spark  session
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("HE Data") \
    .config("spark.driver.extraClassPath", "/content/postgresql-42.3.1.jar") \
    .getOrCreate()

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
spark.sparkContext.addFile("https://expeditiondata.s3.us-east-2.amazonaws.com/updated_data.csv")
updated_df=spark.read.csv(SparkFiles.get('updated_data.csv'),header=True,  inferSchema=True)
updated_df.show()

+-------------+-------+----------+-------+------+-------------+--------------+----------------+-------+-------------+-----------+------------------+------------+---+---+----------------+-------+----+-----------+----+-------------+
|expedition_id|peak_id| peak_name|year_id|season|basecamp_date|highpoint_date|termination_date|members|member_deaths|hired_staff|hired_staff_deaths|   member_id|sex|age|highpoint_metres|success|solo|oxygen_used|died|height_metres|
+-------------+-------+----------+-------+------+-------------+--------------+----------------+-------+-------------+-----------+------------------+------------+---+---+----------------+-------+----+-----------+----+-------------+
|    AMAD78301|   AMAD|Ama Dablam|   1978|Autumn|    10/1/1978|    10/20/1978|      10/23/1978|      8|            0|          5|                 0|AMAD78301-01|  M| 40|              NA|      f|   f|          f|   f|         6814|
|    AMAD78301|   AMAD|Ama Dablam|   1978|Autumn|    10/1/1978|    10/20/197

In [ ]:
#remove highpoint_metres column
# Load in a sql function to use columns
from pyspark.sql.functions import col

In [ ]:
dropped_hm_df = updated_df.drop("highpoint_metres")

In [ ]:
dropped_hm_df.show()

+-------------+-------+----------+-------+------+-------------+--------------+----------------+-------+-------------+-----------+------------------+------------+---+---+-------+----+-----------+----+-------------+
|expedition_id|peak_id| peak_name|year_id|season|basecamp_date|highpoint_date|termination_date|members|member_deaths|hired_staff|hired_staff_deaths|   member_id|sex|age|success|solo|oxygen_used|died|height_metres|
+-------------+-------+----------+-------+------+-------------+--------------+----------------+-------+-------------+-----------+------------------+------------+---+---+-------+----+-----------+----+-------------+
|    AMAD78301|   AMAD|Ama Dablam|   1978|Autumn|    10/1/1978|    10/20/1978|      10/23/1978|      8|            0|          5|                 0|AMAD78301-01|  M| 40|      f|   f|          f|   f|         6814|
|    AMAD78301|   AMAD|Ama Dablam|   1978|Autumn|    10/1/1978|    10/20/1978|      10/23/1978|      8|            0|          5|               

In [ ]:
#create list to filer df to include top 15 most frequented peaks
peak_list=['EVER','CHOY','AMAD','MANA','DHA1',
           'MAKA','LHOT','BARU','PUMO','ANN1','KANG',
           'HIML','ANN4','PUTH','TILI']

In [ ]:
#filter df to only included climbers on peaks in top 15
filtered_df = dropped_hm_df.filter(dropped_hm_df.peak_id.isin(peak_list))

In [ ]:
#row count of filtered df
filtered_df.count()

61638

In [ ]:
# Drop null values
clean_df = filtered_df.dropna()


In [ ]:
#row count after nulls dropped
clean_df.count()

61630

In [ ]:
clean_df.show()

+-------------+-------+----------+-------+------+-------------+--------------+----------------+-------+-------------+-----------+------------------+------------+---+---+-------+----+-----------+----+-------------+
|expedition_id|peak_id| peak_name|year_id|season|basecamp_date|highpoint_date|termination_date|members|member_deaths|hired_staff|hired_staff_deaths|   member_id|sex|age|success|solo|oxygen_used|died|height_metres|
+-------------+-------+----------+-------+------+-------------+--------------+----------------+-------+-------------+-----------+------------------+------------+---+---+-------+----+-----------+----+-------------+
|    AMAD78301|   AMAD|Ama Dablam|   1978|Autumn|    10/1/1978|    10/20/1978|      10/23/1978|      8|            0|          5|                 0|AMAD78301-01|  M| 40|      f|   f|          f|   f|         6814|
|    AMAD78301|   AMAD|Ama Dablam|   1978|Autumn|    10/1/1978|    10/20/1978|      10/23/1978|      8|            0|          5|               

In [ ]:
clean_df.printSchema()

root
 |-- expedition_id: string (nullable = true)
 |-- peak_id: string (nullable = true)
 |-- peak_name: string (nullable = true)
 |-- year_id: integer (nullable = true)
 |-- season: string (nullable = true)
 |-- basecamp_date: string (nullable = true)
 |-- highpoint_date: string (nullable = true)
 |-- termination_date: string (nullable = true)
 |-- members: integer (nullable = true)
 |-- member_deaths: integer (nullable = true)
 |-- hired_staff: integer (nullable = true)
 |-- hired_staff_deaths: integer (nullable = true)
 |-- member_id: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: string (nullable = true)
 |-- success: string (nullable = true)
 |-- solo: string (nullable = true)
 |-- oxygen_used: string (nullable = true)
 |-- died: string (nullable = true)
 |-- height_metres: integer (nullable = true)



In [ ]:
# Configure settings for RDS

server_name= "jdbc:postgresql://group-1.c08lganpj8oa.us-east-2.rds.amazonaws.com:5432/postgres"




In [ ]:
#write data  frame to active
clean_df.write.jdbc(server_name,'clean',  mode='append',properties={
    "user":'root',
    "password":'Group1!!',
    "driver": 'org.postgresql.Driver'
})